# NLP - Classification

For convenience all data cleaning and preparation code is pasted below

**Cleaning**

In [ ]:
# Reading in the data

import pandas as pd
train = pd.read_csv('/Users/helenabelloff/Desktop/NLP/train.csv')
train.head()

# Remove Hyperlinks

def hyperlink_remove(text):
    hyperlink_remove = train['text'].str.replace('http\S+|http.\S+', '', case=False)
    return hyperlink_remove

train['hyperlink_remove'] = hyperlink_remove(train['text'])

# Removing punctuation

import string
string.punctuation

def remove_punct(text):
    nopunct = "".join([char for char in text if char not in string.punctuation])
    return nopunct

train['text_no_punct'] = train['hyperlink_remove'].apply(lambda x: remove_punct(x))

# Remove digits

import string

train['text_no_punct'] = train['text_no_punct'].str.replace(r"\d","", regex= True)

# Text to lower

def text_lower(text):
    text = text.lower()
    return text
train['text_no_punct_lower'] = train['text_no_punct'].apply(lambda x: text_lower(x))

# Getting rid of accents etc.

import unicodedata
train['text_no_punct_lower'] = train['text_no_punct_lower'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

# Trying to standardize words

from itertools import product
import itertools

train['text_no_punct_lower'] = train['text_no_punct_lower'].apply(lambda x:''.join(''.join(s)[:2] for _, s in itertools.groupby(x)))

# Tokenizing

import re

def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

train['tokenized_text'] = train['text_no_punct_lower'].apply(lambda x: tokenize(x.lower()))

stopword = {'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 
            'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 
            'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 
            'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 
            'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 
            'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 
            'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 
            'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 
            'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 
            'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 
            'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 
            'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 
            'was', 'here', 'than', 'amp', 'wa', 'am', 'pm', 'im', 'leh', 'ind', 'inciweb', 'ina'}

def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopword]
    return text

train['tokenized_nostopwords'] = train['tokenized_text'].apply(lambda x: remove_stopwords(x))

# Need to remove stems like "ing" and "ly" - destemming

import nltk

port = nltk.PorterStemmer()

def stemming(tokenized_text):
    text = [port.stem(word) for word in tokenized_text]
    return text

train['tokenized_destemmed'] = train['tokenized_nostopwords'].apply(lambda x: stemming(x))

# Lemmatizing

import nltk

lemmatizer = nltk.WordNetLemmatizer()

def lemmatizing(tokenized_text):
    text = [lemmatizer.lemmatize(word) for word in tokenized_text]
    return text

train['token_lemmatized'] = train['tokenized_nostopwords'].apply(lambda x: lemmatizing(x))

# Joining Text

def join_text(text):
    text = ' '.join(text)
    return text
train['token_lemmatized'] = train['token_lemmatized'].apply(lambda x: join_text(x))

**Preparing the Data**

In [ ]:
# Define body

x = train['token_lemmatized']
print(x.shape)

# Define labels

y = train['target']
print(y.shape)

In [ ]:
# Bag of Words using Tfidf

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import ShuffleSplit
import pandas as pd
from sklearn.model_selection import cross_val_score,train_test_split,GridSearchCV
import numpy as np


cv = CountVectorizer()

# Checking word count
word_count = cv.fit_transform(x)
#print(word_count)

# Ok, word count shape checks out
#print(word_count.shape)

# Now, compute IDF values
tfidf_trans = TfidfTransformer(smooth_idf = True, use_idf = True)
tfidf_trans.fit(word_count)

# Print idf values
idf_df = pd.DataFrame(tfidf_trans.idf_, index = cv.get_feature_names(), columns = ["idf_weights"])
 
# Sort ascending
idf_df.sort_values(by = ['idf_weights'])

count_vector = cv.transform(x)
 
# tf-idf scores
# Computing the tf * idf  multiplication where term frequency is weighted by its IDF values
tfidf_vector = tfidf_trans.transform(count_vector)


feature_names = cv.get_feature_names()

tfidf = TfidfVectorizer()

X1 = cv.fit_transform(x)
X = tfidf.fit_transform(x)
print(X.shape)

train_df = pd.DataFrame(columns = feature_names, data = X.toarray())
print(train_df.shape)
print(train_df.head(10))

# Splitting the data into train and test (validation)

X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,random_state=42)

In [ ]:
# Creating Final Data Frame

train.reset_index(inplace=True, drop = True)
train.drop(['target','text', 'text_no_punct', 'tokenized_text', 'hyperlink_remove', 'tokenized_nostopwords', 
            'tokenized_destemmed', 'token_lemmatized', 'text_no_punct_lower'],axis = 1, inplace = True)
train = pd.concat([train,train_df], axis =1)
print(train.shape)
train.head(10)

In [ ]:
train['keyword'] = train['keyword'].fillna('NA')

# Classification

**Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train,y_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train,y_train)
print("Cross Val Score: ",cross_val_score(rf,X_train,y_train,cv=10).mean())
print("Train Score: ", round(rf.score(X_train,y_train),4))
print("Test Score: ", round(rf.score(X_test,y_test),4))
print(f'Model Accuracy: {rf.score(X, y)}')

**Next Attempt**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create the model with 100 trees
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
model.fit(X_train,y_train)

In [ ]:
print("Cross Val Score: ",cross_val_score(model,X_train,y_train,cv=10).mean())
print("Train Score: ", round(model.score(X_train,y_train),4))
print("Test Score: ", round(model.score(X_test,y_test),4))
print(f'Model Accuracy: {model.score(X, y)}')

**Next - Need to tune the parameters**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create the model with 700 trees
# n_jobs = -1 means we're specifying all processors on your CPUs be used for this purpose (this makes it run the fastest)

model = RandomForestClassifier(n_estimators=700, 
                               bootstrap = True,
                               min_samples_split = 4, 
                               max_features = 'sqrt', 
                               n_jobs = -1, 
                               min_samples_leaf = 1, 
                               criterion = 'gini')
# Fit on training data
model.fit(X_train,y_train)

In [ ]:
print("Cross Val Score: ", cross_val_score(model,X_train,y_train,cv=5).mean())
print("Train Score: ", round(model.score(X_train,y_train),4))
print("Test Score: ", round(model.score(X_test,y_test),4))
print(f'Model Accuracy: {model.score(X, y)}')

In [ ]:
from sklearn.metrics import f1_score
pred = model.predict(X_train)
f_score = f1_score(y_train, pred, average='weighted')
print("F1 Score: ", f_score)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_train, pred))

In [ ]:
from sklearn.metrics import f1_score
pred = model.predict(X_test)
f_score = f1_score(y_test, pred)
print("F1 Score: ", f_score)

In [ ]:
# Confusion Matrices to see distribution of results

from sklearn.metrics import confusion_matrix

pred = model.predict(X_train)
conf_mat = confusion_matrix(y_train, pred)
print(conf_mat)

In [ ]:
from sklearn.metrics import confusion_matrix
pred = model.predict(X_test)
conf_mat = confusion_matrix(y_test, pred)
print(conf_mat)

In [ ]:
# Full classification report

from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

In [ ]:
# Feature Importance

import pandas as pd
feature_imp = pd.Series(model.feature_importances_, index = feature_names).sort_values(ascending = True)
feature_imp

In [ ]:
# Actual class predictions
rf_predictions = model.predict(X_test)
# Probabilities for each class
rf_probs = model.predict_proba(X_test)[:, 1]

In [ ]:
# ROC

from sklearn.metrics import roc_auc_score

# Calculate roc auc
roc_auc = roc_auc_score(y_test, rf_probs)
print(roc_auc)

In [ ]:
# ROC CODE
import matplotlib.pyplot as plt
from sklearn import metrics

probs = model.fit(X_train, y_train).predict_proba(X_test)

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

# Compute ROC curve and area the curve
fpr, tpr, thresholds = roc_curve(y_test, probs[:, 1])
RF_roc_auc = auc(fpr, tpr)
print("Area under the ROC curve for Random Forest : %f" % RF_roc_auc)

In [ ]:
# Plot ROC

import matplotlib.pyplot as pl

# Plot it
pl.clf()
pl.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % RF_roc_auc)
pl.plot([0, 1], [0, 1], 'k--')
pl.xlim([0.0, 1.0])
pl.ylim([0.0, 1.0])
pl.xlabel('False Positive Rate')
pl.ylabel('True Positive Rate')
pl.title('Random Forest ROC')
pl.legend(loc="lower right")
pl.show()

** ExtraTrees **

In [ ]:
# ExtraTreesClassifier

from sklearn.ensemble import ExtraTreesClassifier

ExtraTreesModel = ExtraTreesClassifier(n_estimators = 700, 
                                       min_samples_split=2, 
                                       n_jobs = -1, max_features = 'sqrt')
ExtraTreesModel.fit(X_train,y_train)

In [ ]:
print("Train Score: ", round(ExtraTreesModel.score(X_train,y_train),4))
print("Test Score: ", round(ExtraTreesModel.score(X_test,y_test),4))

In [ ]:
# Actual class predictions
rf_predictions = ExtraTreesModel.predict(X_test)
# Probabilities for each class
rf_probs = ExtraTreesModel.predict_proba(X_test)[:, 1]

In [ ]:
from sklearn.metrics import roc_auc_score

# Calculate roc auc
roc_auc = roc_auc_score(y_test, rf_probs)
print(roc_auc)

** AdaBoost **

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

adaModel = AdaBoostClassifier(n_estimators = 1000, learning_rate = 1, random_state = 1)
adaModel.fit(X_train,y_train)

In [ ]:
print("Cross Val Score: ", cross_val_score(adaModel,X_train,y_train,cv=10).mean())
print("Train Score: ", round(adaModel.score(X_train,y_train),4))
print("Test Score: ", round(adaModel.score(X_test,y_test),4))
print(f'Model Accuracy: {adaModel.score(X, y)}')

In [ ]:
y_pred_ada = adaModel.predict(X_test)

In [ ]:
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_ada))

In [ ]:
# Confusion Matrices
from sklearn.metrics import confusion_matrix

pred = adaModel.predict(X_train)
conf_mat = confusion_matrix(y_train, pred)
print(conf_mat)

In [ ]:
# Confusion Matrices
from sklearn.metrics import confusion_matrix

pred = adaModel.predict(X_test)
conf_mat = confusion_matrix(y_test, pred)
print(conf_mat)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

In [ ]:
# ROC CODE
import matplotlib.pyplot as plt
from sklearn import metrics

probs = adaModel.fit(X_train, y_train).predict_proba(X_test)

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

# Compute ROC curve and area the curve
fpr, tpr, thresholds = roc_curve(y_test, probs[:, 1])
Ada_roc_auc = auc(fpr, tpr)
print("Area under the ROC curve for AdaBoost : %f" % Ada_roc_auc)

In [ ]:
import matplotlib.pyplot as pl

# Plot it
pl.clf()
pl.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % Ada_roc_auc)
pl.plot([0, 1], [0, 1], 'k--')
pl.xlim([0.0, 1.0])
pl.ylim([0.0, 1.0])
pl.xlabel('False Positive Rate')
pl.ylabel('True Positive Rate')
pl.title('AdaBoost ROC')
pl.legend(loc="lower right")
pl.show()

In [ ]:
import pandas as pd
feature_imp = pd.Series(adaModel.feature_importances_, index = feature_names).sort_values(ascending = False)
feature_imp

** AdaBoost with SVC as base estimator **

You will see this is very bad for accuracy...

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn import metrics

svc = SVC(probability = True, kernel = 'linear')

AdaBoostModelSVC = AdaBoostClassifier(n_estimators = 100, base_estimator = svc, learning_rate = 1)

AdaBoostModelSVC.fit(X_train,y_train)

In [ ]:
print("Train Score: ", round(AdaBoostModelSVC.score(X_train,y_train),4))
print("Test Score: ", round(AdaBoostModelSVC.score(X_test,y_test),4))
print(f'Model Accuracy: {AdaBoostModelSVC.score(X, y)}')

**Support Vector Machine**

In [ ]:
from sklearn.svm import SVC

SVCmodel = SVC(random_state=1, kernel='linear')

SVCmodel.fit(X_train,y_train)

In [ ]:
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto', probability=True)
SVM.fit(X_train,y_train)

predictions_SVM = SVM.predict(X_test)

print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM, y_test)*100, "%")

In [ ]:
print("Cross Val Score: ", cross_val_score(SVCmodel,X_train,y_train,cv=10).mean())
print("Train Score: ", round(SVCmodel.score(X_train,y_train),4))
print("Test Score: ", round(SVCmodel.score(X_test,y_test),4))

In [ ]:
y_pred = SVCmodel.predict(X_test)

In [ ]:
from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
# Confusion Matrices
from sklearn.metrics import confusion_matrix

pred = SVM.predict(X_train)
conf_mat = confusion_matrix(y_train, pred)
print(conf_mat)

In [ ]:
# Confusion Matrices
from sklearn.metrics import confusion_matrix

pred = SVM.predict(X_test)
conf_mat = confusion_matrix(y_test, pred)
print(conf_mat)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

In [ ]:
# ROC CODE
import matplotlib.pyplot as plt
from sklearn import metrics

probs = SVM.fit(X_train, y_train).predict_proba(X_test)

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

# Compute ROC curve and area the curve
fpr, tpr, thresholds = roc_curve(y_test, probs[:, 1])
SVM_roc_auc = auc(fpr, tpr)
print("Area under the ROC curve for SVM : %f" % SVM_roc_auc)

In [ ]:
import matplotlib.pyplot as pl

# Plot ROC curve
pl.clf()
pl.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % SVM_roc_auc)
pl.plot([0, 1], [0, 1], 'k--')
pl.xlim([0.0, 1.0])
pl.ylim([0.0, 1.0])
pl.xlabel('False Positive Rate')
pl.ylabel('True Positive Rate')
pl.title('SVM ROC')
pl.legend(loc="lower right")
pl.show()

** SVM with Sigmoid Kernel **

You will see that the sigmoid kernel is not appopriate for this data

In [ ]:
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

SVM = svm.SVC(C= 0.5, kernel='sigmoid', degree=3, gamma='auto', probability=True)
SVM.fit(X_train,y_train)

predictions_SVM = SVM.predict(X_test)

print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM, y_test)*100, "%")

In [ ]:
print("Train Score: ", round(SVM.score(X_train,y_train),4))
print("Test Score: ", round(SVM.score(X_test,y_test),4))